In [17]:
import json
import pandas as pd
import requests

In [18]:
url = 'https://prices.runescape.wiki/api/v1/osrs/latest'
headers = {
    'User-Agent': 'ge_tracking',
    'From': 'onlypking'
}

response = requests.get(url, headers = headers)

In [19]:
response.json()

{'data': {'2': {'high': 182,
   'highTime': 1649387659,
   'low': 175,
   'lowTime': 1649387590},
  '6': {'high': 183629,
   'highTime': 1649386136,
   'low': 179367,
   'lowTime': 1649386909},
  '8': {'high': 189499,
   'highTime': 1649386996,
   'low': 178502,
   'lowTime': 1649386841},
  '10': {'high': 189989,
   'highTime': 1649387237,
   'low': 182567,
   'lowTime': 1649387061},
  '12': {'high': 198200,
   'highTime': 1649387025,
   'low': 186285,
   'lowTime': 1649387059},
  '28': {'high': 250,
   'highTime': 1649369794,
   'low': 150,
   'lowTime': 1649383778},
  '30': {'high': 677,
   'highTime': 1649387365,
   'low': 611,
   'lowTime': 1649366203},
  '36': {'high': 100,
   'highTime': 1649387418,
   'low': 100,
   'lowTime': 1649386932},
  '39': {'high': 4, 'highTime': 1649387144, 'low': 4, 'lowTime': 1649380339},
  '40': {'high': 10, 'highTime': 1649386479, 'low': 7, 'lowTime': 1649383650},
  '41': {'high': 22, 'highTime': 1649384994, 'low': 22, 'lowTime': 1649387657},
  '42'

In [20]:
json_resp = response.json()

In [21]:
json_resp['data']['2']

{'high': 182, 'highTime': 1649387659, 'low': 175, 'lowTime': 1649387590}

In [22]:
df = pd.DataFrame.from_dict(json_resp['data'])

In [23]:
df

,2,6,8,10,12,28,30,36,39,40,...,26528,26541,26554,26557,26560,26788,26798,26801,26804,26815
high,182,183629,189499,189989,198200,250,677,100,4,10,...,1079999,50000,473476,1500000,8000000,499,15431,12557,3999,4600000
highTime,1649387659,1649386136,1649386996,1649387237,1649387025,1649369794,1649387365,1649387418,1649387144,1649386479,...,1649387645,1649381681,1649367727,1649295493,1649385957,1649387516,1649383929,1649387280,1649387370,1649386325
low,175,179367,178502,182567,186285,150,611,100,4,7,...,1005500,38621,1100000,1001111,8000000,327,14000,3003,1000,4201003
lowTime,1649387590,1649386909,1649386841,1649387061,1649387059,1649383778,1649366203,1649386932,1649380339,1649383650,...,1649387410,1649387082,1649293318,1649367622,1649385958,1649387496,1649386563,1649382074,1649386025,1649387013


In [24]:
trans_df = df.transpose()

In [25]:
trans_df

,high,highTime,low,lowTime
2,182.0,1.649388e+09,175.0,1.649388e+09
6,183629.0,1.649386e+09,179367.0,1.649387e+09
8,189499.0,1.649387e+09,178502.0,1.649387e+09
10,189989.0,1.649387e+09,182567.0,1.649387e+09
12,198200.0,1.649387e+09,186285.0,1.649387e+09
...,...,...,...,...
26788,499.0,1.649388e+09,327.0,1.649387e+09
26798,15431.0,1.649384e+09,14000.0,1.649387e+09
26801,12557.0,1.649387e+09,3003.0,1.649382e+09
26804,3999.0,1.649387e+09,1000.0,1.649386e+09


In [26]:
clean_df = trans_df.drop(['highTime', 'lowTime'], axis = 1)
clean_df

,high,low
2,182.0,175.0
6,183629.0,179367.0
8,189499.0,178502.0
10,189989.0,182567.0
12,198200.0,186285.0
...,...,...
26788,499.0,327.0
26798,15431.0,14000.0
26801,12557.0,3003.0
26804,3999.0,1000.0


In [29]:
new = clean_df.rename_axis('item')
new

,high,low
item,,
2,182.0,175.0
6,183629.0,179367.0
8,189499.0,178502.0
10,189989.0,182567.0
12,198200.0,186285.0
...,...,...
26788,499.0,327.0
26798,15431.0,14000.0
26801,12557.0,3003.0
